In [28]:
from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin
import numpy as np

class MyOwnRegressor(BaseEstimator, RegressorMixin):
    
    """
    An example of regressor
    """
    
    def __init__(self, c=1):
        """
        Called when initializing the classifier
        """
        self.c = c
    
    def fit(self, X, y):
        return self
    
    def predict(self, X):
        return np.mean(X, axis=1)/self.c

In [29]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
print(X.shape)

(506, 13)


In [42]:
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LinearRegression

# define search space
parameters = {'c':[1, 2, 3, 5, 6, 7]}

# define evaluation
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define the search
search = BayesSearchCV(estimator=reg, search_spaces=parameters, n_jobs=-1, n_iter=10
#                        cv=cv
                      )

# perform the search
search.fit(X, y)
# report the best result
print(search.best_score_)
print(search.best_params_)

-1.615666114220661
OrderedDict([('c', 3)])


In [43]:
search.best_params_

OrderedDict([('c', 3)])

In [3]:
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV

params = dict()
params['C'] = (1e-6, 100.0, 'log-uniform')
params['gamma'] = (1e-6, 100.0, 'log-uniform')
params['degree'] = (1,5)
params['kernel'] = ['linear', 'sigmoid']
# define evaluation
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the search
search = BayesSearchCV(estimator=SVR(), n_iter=2, search_spaces=params, n_jobs=-1,
#                        cv=cv
                      )
# perform the search
search.fit(X, y)

BayesSearchCV(estimator=SVR(), n_iter=2, n_jobs=-1,
              search_spaces={'C': (1e-06, 100.0, 'log-uniform'),
                             'degree': (1, 5),
                             'gamma': (1e-06, 100.0, 'log-uniform'),
                             'kernel': ['linear', 'sigmoid']})

In [4]:
search.

0.5007020255538938

In [ ]:
import BR_ML as br
import pandas as pd

In [ ]:
print(br.__version__)

In [ ]:
pequim = pd.read_csv('datasets/BeijingPM20100101_20151231.csv')
pequim.columns

In [ ]:
black_friday = pd.read_csv('datasets/BlackFriday.csv')
black_friday.columns                           

In [ ]:
fb = pd.read_csv('datasets/FB.csv')
fb.columns

In [ ]:
fb['Date'] = pd.to_datetime(fb['Date'])
fb = fb.set_index('Date')

In [ ]:
fb_outliers = br.outlier.outlier_detection(fb, 'LOCI')
fb_outliers.head()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fb_outliers['Volume'])
plt.plot(fb_outliers[fb_outliers['Volume_outlier_flag'] == 1]['Volume'], 'o')

In [ ]:
fb_sem_outliers = br.outlier.outlier_treatment(fb_outliers, method='mean')

In [ ]:
plt.plot(fb['Volume'])
plt.plot(fb_sem_outliers['Volume'])

In [ ]:
pequim_sem_outliers.head()

In [ ]:
pequim_outliers[(pequim_outliers['precipitation_outlier_flag'] == 1) | 
                (pequim_outliers['Iprec_outlier_flag'] == 1)].shape

In [ ]:
pequim_outliers.shape[0] - pequim_sem_outliers.shape[0]

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV

In [ ]:
X = fb.iloc[:, :-1]
y = fb.iloc[:, -1]

In [ ]:
# define search space
params = dict()
params['order'] = [(1,0,0), (0,1,0), ((0,0,1))]
params['seasonal_order'] = [(1,0,0,0), (0,1,0,0), ((0,0,1,0))]

In [ ]:
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
# define the search
search = BayesSearchCV(estimator=SARIMAX(y.values), search_spaces=params, n_jobs=-1, cv=cv)

In [ ]:
search.fit()

In [ ]:

# perform the search
search.fit(X, y)
# report the best result
print(search.best_score_)
print(search.best_params_)